In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

In [2]:
dc={
    "MEMHI":[(0,50),(51,100),(101,200),(201,300),(301,400)],
    "condition":["Very Good","Good","normal","unhealthy","severe"],
    "CO":[(0,12),(13,22),(23,30),(31,49),(50,4000000000)],
    "H2S":[(0,3),(3.1,5),(5.1,12.9),(13,20),(20,4000000000)],
    "CH4":[(0,1000),(1001,2000),(2001,4000),(4001,5000),(5001,4000000000)],
    "SPO2":[(98,100),(96,97),(93,95),(90,92),(0,89)],

}

In [3]:
df=pd.DataFrame(dc)

In [4]:
df.head()

,MEMHI,condition,CO,H2S,CH4,SPO2
0,"(0, 50)",Very Good,"(0, 12)","(0, 3)","(0, 1000)","(98, 100)"
1,"(51, 100)",Good,"(13, 22)","(3.1, 5)","(1001, 2000)","(96, 97)"
2,"(101, 200)",normal,"(23, 30)","(5.1, 12.9)","(2001, 4000)","(93, 95)"
3,"(201, 300)",unhealthy,"(31, 49)","(13, 20)","(4001, 5000)","(90, 92)"
4,"(301, 400)",severe,"(50, 4000000000)","(20, 4000000000)","(5001, 4000000000)","(0, 89)"


In [5]:
def calculate_CO(x):
    for i,index in zip(df["CO"],df['MEMHI']):
        if x>=i[0] and x<=i[1]:
            if 4000000000 in i:
                return index,(i[0],i[0]+1)
            return index,i

def calculate_H2S(x):
    for i,index in zip(df["H2S"],df['MEMHI']):
        if x>=i[0] and x<=i[1]:
            if 4000000000 in i:
                return index,(i[0],i[0]+1)
            return index,i

def calculate_CH4(x):
    for i,index in zip(df["CH4"],df['MEMHI']):
        if x>=i[0] and x<=i[1]:
            if 4000000000 in i:
                return index,(i[0],i[0]+1)
            return index,i

def calculate_SPO2(x):
    for i,index in zip(df["SPO2"],df['MEMHI']):
        if x>=i[0] and x<=i[1]:
            return index,(i[1],i[0])

In [6]:
def subIndexFormulaOne(gas,value):
    ilow,ihigh,blow,bhigh=gas[0][0],gas[0][1],gas[1][0],gas[1][1]
    print(ilow,ihigh,blow,bhigh)
    return round(((ihigh-ilow)/(bhigh-blow))*(value-blow)+ilow)

In [7]:
# Root-Sum-Power Form (non-linear aggregation form)
def subIndexFormulaTwo(co,h2s,ch4,spo2,power):
    return round(np.power(co**power+h2s**power+ch4**power+spo2**power,1/power))

In [8]:
def calculate_index(x,power):
    co=calculate_CO(x["CO"])
    h2s=calculate_H2S(x["H2S"])
    ch4=calculate_CH4(x["CH4"])
    spo2=calculate_SPO2(x["SPO2"])
    co=subIndexFormulaOne(co,x["CO"])
    h2s=subIndexFormulaOne(h2s,x["H2S"])
    ch4=subIndexFormulaOne(ch4,x["CH4"])
    spo2=subIndexFormulaOne(spo2,x["SPO2"])
    return subIndexFormulaTwo(co,h2s,ch4,spo2,power)
    

In [9]:
x={
    "CO":13,
    "H2S":3,
    "CH4":1000,
    "SPO2":98
}

In [10]:
calculate_index(x,9)

51 100 13 22
0 50 0 3
0 50 0 1000
0 50 100 98


59

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
co_model=tf.keras.models.load_model("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/models/CO_lstm1.h5")
ch4_model=tf.keras.models.load_model("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/models/CH4_lstm1.h5")
h2s_model=tf.keras.models.load_model("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/models/H2S_lstm1.h5")
spo2_model=tf.keras.models.load_model("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/models/SPO2_lstm1.h5")

In [14]:
# encoder loading
with open("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/encoders/CO_encoder1.pkl","rb") as f:
    co_encoder=pickle.load(f)
with open("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/encoders/CH4_encoder1.pkl","rb") as f:
    ch4_encoder=pickle.load(f)
with open("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/encoders/H2S_encoder1.pkl","rb") as f:
    h2s_encoder=pickle.load(f)
with open("/content/drive/MyDrive/freelancing/ehtisham work/Index prediction/encoders/SPO2_encoder1.pkl","rb") as f:
    spo2_encoder=pickle.load(f)


In [ ]:
def index_prediction(x,model,encoder):
  data={
        "date":[date],
        "time":[time],
    }
    data=pd.DataFrame(data)
    print(data)
    # data['date'] = data['date'].dt.strftime('%Y-%m-%d')
    data['DateTime']=data['date']+' '+data["time"]
    data['DateTime'] = pd.to_datetime(data['DateTime'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
    data.drop(["date","time"],axis=1,inplace=True)
    prediction=model.predict(data)
    prediction=encoder.inverse_transform(prediction)
    return prediction
    